In [1]:
import json
import re
import os

import boto3
import awswrangler as wr
import pandas as pd
import numpy as np

from dataclasses import dataclass

# Define a class for the ship particulars

In [2]:
@dataclass
class ShipTechnicalSpecs:
    IMO_number: str
    # call_sign: str
    # mmsi: str
    # flag: str
    built_year: str
    length: str
    beam: str
    # depth: str
    # draft: str
    gross_tonnage: str
    # net_tonnage: str
    # engines: str
    # propulsion: str
    # speed: str
    dwt: str
    # capacity: str

In [64]:
def extract_field(value, field_name):
    pattern = {
        "IMO": r"IMO number:\s*(\d+)",
        "Call Sign": r"Call\s?sign[:\s]*([A-Za-z0-9]+)",
        "MMSI": r"MMSI number[:\s]*([A-Za-z0-9]+)",
        "GT": r"(\d{1,3}(?:,\d{3})*(?:\.\d+)?)[\s\u00A0]*(?:GT|GRT)",
        "NT": r"(\d{1,3}(?:,\d{3})*(?:\.\d+)?)[\s\u00A0]*NT",
        "DWT": r"(\d{1,3}(?:,\d{3})*(?:\.\d+)?)[\s\u00A0]*(?:t[\s\u00A0]*)?DWT"
    }
    match = re.search(pattern.get(field_name, ""), value, re.IGNORECASE)
    if match:
        return match.group(1).replace(",", "")  # Remove commas from the numeric value
    return "N/A"

In [35]:
def get_year(ship_details, name1:str, name2:str):
    year_of_completion = ship_details.get(name1, "")
    if year_of_completion:
        return year_of_completion
    
    return ship_details.get(name2, "Missing")

In [8]:
def get_similar_values(ship_details, name1:str, name2:str):
    beam = ship_details.get(name1, "")
    if beam:
        match = re.search(r"\d+\.?\d*\s*m", beam, re.IGNORECASE)
        if match:
            return match.group(0)
    
    # Try Breadth dictionary format
    return ship_details.get(name2, "Missing")

In [5]:
def get_tonnage_value(ship_details, primary_field, backup_field, regex_pattern):
    direct_value = ship_details.get(backup_field)
    if direct_value and direct_value != "Missing":
        return str(direct_value)
    
    tonnage = ship_details.get("Tonnage", "")
    if tonnage:
        extracted = extract_field(tonnage, regex_pattern)
        if extracted != "N/A":
            return extracted
            
    return "Missing"

In [65]:
def process_ship_data(ship_data):
    processed_data = []

    for imo_number, ship_details in ship_data.items():
        imo_number = imo_number
        
        identification = ship_details.get("Identification", "Missing")
        # mmsi = extract_field(identification, "MMSI")
        # call_sign = extract_field(identification, "Call Sign")
        
        # flag = ship_details.get("Flag", "Missing")
        # built_year = ship_details.get("Built year", "Missing")
        built_year = get_year(ship_details, "Completed", "Built year")
        # displacement = ship_details.get("Displacement", "Missing")
        # length = ship_details.get("Length", "Missing")
        length = get_similar_values(ship_details, "Length", "Length (m)")
        # beam = ship_details.get("Beam", "Missing") or ship_details.get("Breadth", "Missing")
        beam = get_similar_values(ship_details, "Beam", "Breadth (m)")
        # breadth = ship_details.get("Breadth", "Missing")
        # depth = ship_details.get("Depth", "Missing")
        # draft = ship_details.get("Draft", "Missing") or ship_details.get("Draught", "Missing")
        # draft = get_similar_values(ship_details, "Draft", "Draught")
        # draught = ship_details.get("Draught", "Missing")
        
        # tonnage = ship_details.get("Tonnage", "Missing")
        # gross_tonnage = extract_field(tonnage, "GT") or ship_details.get("grossTonnage", "Missing")
        # net_tonnage = extract_field(tonnage, "NT") or ship_details.get("netTonnage", "Missing")
        # deadweight = extract_field(tonnage, "DWT") or ship_details.get("DWT", "Missing")
        
        gross_tonnage = get_tonnage_value(ship_details, "Tonnage", "grossTonnage", "GT")
        # net_tonnage = get_tonnage_value(ship_details, "netTonnage", "NT", "NT")
        deadweight = get_tonnage_value(ship_details, "DWT", "deadweight", "DWT")
        
        # engines = ship_details.get("Installed power", "Missing")
        # propulsion = ship_details.get("Propulsion", "Missing")
        # speed = ship_details.get("Speed", "Missing")
        # capacity = ship_details.get("Capacity", "Missing")
        
        ship = ShipTechnicalSpecs(
            IMO_number=imo_number,
            # mmsi=mmsi,
            # call_sign=call_sign,
            # flag=flag,
            built_year=built_year,
            length=length,
            beam=beam,
            # depth=depth,
            # draft=draft,
            gross_tonnage=gross_tonnage,
            # net_tonnage=net_tonnage,
            dwt=deadweight,
            # engines=engines,
            # propulsion=propulsion,
            # speed=speed,
            # capacity=capacity
        )
        processed_data.append(ship)
    return processed_data

In [90]:
file_names = ["wikipedia_ship_data_v2", "pleiades_fleet_v2"]
dataframes = []

for name in file_names:
    with open(f'../data/raw/ship_particulars/{name}.json', 'r') as f:
        data = json.load(f)
        processed_data = process_ship_data(data)
        output_data = [ship.__dict__ for ship in processed_data]
        dataframes.append(pd.DataFrame(output_data))

In [91]:
df = pd.concat(dataframes, ignore_index=True)

In [92]:
df

,IMO_number,built_year,length,beam,gross_tonnage,dwt
0,1007213,2003,126.20 m,21.00 m,9932,Missing
1,6602898,1966,135 m,15.8 m,10417,Missing
2,7037806,1971,123.40 m,19.20 m,9149,Missing
3,7207451,February 1975,114.59 m,18.62 m,4649,Missing
4,7350090,6 September 1974,118 m,20 m,4774,Missing
...,...,...,...,...,...,...
478,9912854,2022,249.9,44.0,62809,Missing
479,9923009,2022,249.9,44.0,62568,Missing
480,9930624,2023,249.9,44.0,62694,Missing
481,9935143,2023,249.9,44.0,62955,Missing


In [93]:
df['dwt'].value_counts(dropna=False)

Missing    264
2700         5
54549        5
185000       4
156907       4
          ... 
5493         1
680          1
4700         1
4768         1
106045       1
Name: dwt, Length: 177, dtype: int64

In [102]:
df[df['dwt'] == 'Missing'][['IMO_number', 'dwt']]

,IMO_number,dwt
0,1007213,Missing
1,6602898,Missing
2,7037806,Missing
3,7207451,Missing
4,7350090,Missing
...,...,...
478,9912854,Missing
479,9923009,Missing
480,9930624,Missing
481,9935143,Missing


In [89]:
df[df['IMO_number'] == '9384992']

,IMO_number,built_year,length,beam,gross_tonnage,dwt
471,9384992,Missing,230.0 m,42.0 m,57462,106045


In [100]:
import time, random

In [ ]:
df['beam'].value_counts(dropna=False)

In [13]:
regex_pattern = r'(\d+(\.\d+)?)\s*m'
df['length (m)'] = df['length'].str.extract(regex_pattern)[0]
df['beam (m)'] = df['beam'].str.extract(regex_pattern)[0]
df['depth (m)'] = df['depth'].str.extract(regex_pattern)[0]
df['draft (m)'] = df['draft'].str.extract(regex_pattern)[0]

In [ ]:
df

In [14]:
df.drop(['length', 'beam', 'depth', 'draft'], axis=1, inplace=True)

In [18]:
df.replace('Missing', np.nan, inplace=True)

In [19]:
df.replace('N/A', np.nan, inplace=True)

In [ ]:
df

In [21]:
df['IMO_number'] = df['IMO_number'].astype(int)

In [ ]:
df.dtypes

In [24]:
import sdv
from sdv.metadata import Metadata

In [25]:
metadata = Metadata.detect_from_dataframe(
    data=df,
    table_name='ship_specs')

In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer = GaussianCopulaSynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(df)

# Step 3: Generate synthetic data
synthetic_data = synthesizer.sample(num_rows=100)

In [ ]:
synthetic_data.head()